In [501]:
%matplotlib notebook
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

import numpy as np
import math as m

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.pyplot as plt
from matplotlib import ticker

## Step 1: Function evaluations 
Evaluation of the objective function and its gradient given a specific point $p$, number of segments $n$, segment lenghts $l$ and a set of configuration angles $v$. 

In [546]:
def evalu(n,l,p0,v):
    
    temp0 = np.zeros(n)
    temp1 = np.zeros(n)
    temp2 = np.zeros(n)
    
    f = np.zeros(2)
    g = np.zeros(2)
    ggh = np.zeros((3,2))
    grad_g = np.zeros((3,2))
    
    for i in range(n):
        
        c = m.cos(v[i]+np.sum(temp0))
        s = m.sin(v[i]+np.sum(temp0))
        
        f[0] = (l[i]*c)+f[0]
        f[1] = (l[i]*s)+f[1]
        
        ggh[0,0] = l[i]*(-s)+ggh[0,0]
        ggh[0,1] = l[i]*(c)+ggh[0,1]
        
        temp0[i] = v[i]
        
        
    for i in range(1,n):
        
        temp1[i] = v[i-1]
        
        c = m.cos(v[i]+np.sum(temp1))
        s = m.sin(v[i]+np.sum(temp1))
        
        ggh[1,0] = l[i]*(-s)+ggh[1,0]
        ggh[1,1] = l[i]*(c)+ggh[1,1]
        
        
        
    for i in range(2,n):
        
        temp2[i-1] = v[i-2]
        temp2[i] = v[i-1]
        
        c = m.cos(v[i]+np.sum(temp2))
        s = m.sin(v[i]+np.sum(temp2))
        
        ggh[2,0] = l[i]*(-s)+ggh[2,0]
        ggh[2,1] = l[i]*(c)+ggh[2,1]
        
        
        
    
    g[0] = 0.5*(f[0]-p0[0])**2
    g[1] = 0.5*(f[1]-p0[1])**2
    
    grad_g = ggh @ (f-p)
    
#     for i in range(n):
        
#         for j in range(2):
            
#             if j==0:
# #                 grad_g[i,j] = ggh[i,j]@(f[0]-p0[0])
#                 grad_g = ggh @ ()
#             else:
#                 grad_g[i,j] = ggh[i,j]@(f[1]-p0[1])
            
    
    return (g,grad_g)



        
        
        




## Step 2:Line search
The first thing we will do is to find the line search using "Backtracking Armijo".

In [547]:
def BacktrackLineSearch(vk, gk, ggk, pk, ak, c, rho, nmaxls=100):
    
    pkggk = pk @ ggk

   
    g,ggk = evalu(n,l,p0,vk)
    
    while g[0] <= gk[0] + c*ak*pkggk and g[1] <= gk[1] + c*ak*pkggk:
        ak *= rho
        vk = vk + np.multiply(ak, pk)
        g,ggk = evalu(n,l,p,vk)
        
    return ak

## Step 2:
Then, we need to compute the search direction. For Gradient descent, this is simply $p_{k+1}=-\nabla f(x_{k+1})$, and we have already computed this during the first step.

## Step 3:Optimization
Now we need to compute the search direction, perform line search along this direction and then perform the step.

In [548]:
def optimize(n, l, p0, c, rho, tol):
    
    vk = v0
    v_temp = np.zeros((nmax,3))
    listofAng = np.zeros((nmax,3))
    gk,ggk = evalu(n,l,p0,vk)
    pk = None
    ak = 1
    
    
    
    for k in range(nmax):
        
        pk = -ggk   #The search direction for steepest descent is simply the gradient
        ak = BacktrackLineSearch(vk, gk, ggk, pk, ak, c, rho)
        
    
        v_temp = np.append(vk, vk + np.multiply(ak, pk), axis=0)
        listofAng[k] = vk + np.multiply(ak, pk)
        vk = listofAng[k]
        ggkold = ggk
        gk,ggk = evalu(n,l,p0,vk) 
        
        if np.linalg.norm(ggk) < tol:
            break
    
    return listofAng
    

In [563]:
#Input arguments
n = 3  #Number of segments
l = np.array([3, 2, 2])
p0 = np.array([3, 2])

#Initial conditions
#v0 = np.array([1.72, -1.54, -0.9])
v0 = np.array([0.78, 0.78, 0.78])

#Algorithm parameters
c = 0.1
rho = 0.5
tol = 1e-8
nmax = 70

vgd = optimize(n, l, p0, c, rho, tol)
topi = 2*m.pi


(a, b) = vgd.shape

for i in range(a):
    for j in range(b):
        if vgd[i,j]>topi:
            fac = int(vgd[i,j]/(topi))
            vgd[i,j] = vgd[i,j]-(topi*fac)
        elif vgd[i,j]<0:
            fac = int(vgd[i,j]/(topi))
            vgd[i,j] = vgd[i,j]+(topi*fac)
            vgd[i,j] = -vgd[i,j]
            
print(vgd)
            









[[2.68999121e+01 2.05018108e+00 2.49972593e+00]
 [1.30236234e+01 4.82975167e+00 1.74609314e+00]
 [6.26387134e+00 1.43820305e+01 5.60251889e+00]
 [5.06632455e+00 1.83063907e+01 2.84772071e-02]
 [4.12874665e+00 1.71425091e+00 3.90938123e+00]
 [8.48321107e-02 2.01155969e+00 1.88116607e+00]
 [6.15661914e+00 4.19924991e+00 3.36506106e+00]
 [5.51687649e+00 3.27740996e+00 4.35822080e+00]
 [3.04312152e+00 1.56779845e+00 1.94286022e-01]
 [1.81306129e+00 2.34458622e-01 6.35649196e-01]
 [3.36050049e+00 5.37240290e+00 1.35462529e+00]
 [5.65495074e+00 3.12258303e+00 2.09333713e+00]
 [1.87938425e+00 2.91146445e+00 5.76013372e+00]
 [1.47365082e+00 5.80889796e+00 5.27897557e+00]
 [5.26847471e-01 4.52301553e+00 6.24177850e+00]
 [2.51353242e+00 3.98140604e-01 8.58075047e-01]
 [6.24722291e+00 3.45726191e+00 3.59625621e+00]
 [3.04393907e+00 2.53297948e+00 4.96615198e+00]
 [4.96259023e-01 3.32440149e+00 4.08451637e+00]
 [2.22935461e+00 2.10809008e+00 6.60691601e-02]
 [5.80061133e-02 1.97210162e+00 3.487668

In [495]:
###############################################################################################################################

###  Debugging 

############################################################################################################################





# vk = v0

# pk = -ggk
# print("This is ggk:",ggk)
# print(ggk.shape)

# print("This is pk:",pk)
# print(pk.shape)


ak = 0.5
p0 = np.array([3, 2])
vk = [m.pi, m.pi, m.pi]
gk,ggk = evalu(n,l,p0,vk) 
pk = -ggk
w = np.multiply(ak, pk)
vk = vk + w
print(pk)
print(w)
print(vk)

    
#     while g[0] <= gk[0] + c*ak*pkggk and g[1] <= gk[1] + c*ak*pkggk:
#         ak *= rho
#         vk = vk + np.multiply(ak, pk)
#         g,ggk = evalu(n,l,p, vk)
        
#     return ak



[-6.00000000e+00 -1.46957616e-15 -4.00000000e+00] 
[-3.00000000e+00 -7.34788079e-16 -2.00000000e+00] 
[0.14159265 3.14159265 1.14159265] 
